In [ ]:
from Dataset import DatasetBuilder

start_directory = "C:\\Users\\gciprianherrera\\Desktop\\LLM\\MVP_Chatbot\\PDF"
pdf_processor = DatasetBuilder(start_directory)
data = pdf_processor.process_directory()

In [3]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings

In [4]:
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model=os.getenv("AZURE_OPENAI_MODEL"),
    temperature=0,
)

In [3]:
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_EMB_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_EMB_API_KEY"),
    openai_api_version=os.getenv("AZURE_OPENAI_EMB_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_EMB_DEPLOYMENT_NAME"),
)

In [6]:
index_name: str = "complete_dataset"
vector_store: AzureSearch = AzureSearch(
    azure_search_key = os.getenv("AZURE_AI_SEARCH_API_KEY"),
    azure_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    index_name=index_name,
    embedding_function=embeddings.embed_query
)

In [7]:
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(data, page_content_column="page_content")
documents = loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')
def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=5,
    length_function=tiktoken_len,
    separators=["\n", " ", ""])
splits = text_splitter.split_documents(documents)
vector_store.add_documents(documents=splits)

In [9]:
AZURE_AI_SEARCH_API_KEY=os.getenv("AZURE_AI_SEARCH_API_KEY")
AZURE_AI_SEARCH_ENDPOINT=os.getenv("AZURE_AI_SEARCH_ENDPOINT")
AZURE_AI_SEARCH_DEPLOYEMENT_ID=os.getenv("AZURE_AI_SEARCH_DEPLOYEMENT_ID")
AZURE_AI_SEARCH_INDEX_NAME=os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
AZURE_AI_SEARCH_SERVICE_NAME=os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")

In [101]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_openai import AzureChatOpenAI
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough
# from langchain import hub
# from langchain_core.runnables import RunnableParallel
from langchain_community.retrievers import (
    AzureAISearchRetriever,   
)

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-05-15",
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model="gpt-35-turbo",
    temperature=0,
)

index_name: str = "complete_dataset"
retriever = AzureAISearchRetriever(
    content_key="content", top_k=30, index_name=index_name
)

# query = "Quel est l'objectif du compte-rendu de la cellule 951 DEG"  
# query = "Quel est l'objectif de l'investigation sur l’ensemble porte-lame cellule 904"  
# query = "Quels sont les équipements investigués sur le périmètre DEG"  
# query = "Quel est le débit de dose de la 951 du DEG " 
# query = "Quelles sont les informations de la cellule 909 du DEG " 
# query = "Quelle est la quantité de boue de la fosse 217-02 du DEG"  
# query = "DDD des PE de boues sur la fosse 217-02 du DEG " 
query = "Quels sont les enseignements sur les investigations de la cellule 959B du HADE"
# query = "Quelles sont les informations sur l'état de la cellule 929A du périmètre HADE"

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use five sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)
result = chain.invoke({"input": query})


import json
from collections import OrderedDict

def extract_list_sources(result):
    sources = []
    for doc in result['context']:
        try:
            data = doc.metadata['metadata']
            data = json.loads(data)
            source = data.get('Source')
            if source:
                sources.append(source)
        except json.JSONDecodeError:
            # Handle the case where metadata is not a valid JSON
            continue
    sources = list(OrderedDict.fromkeys(sources))
    return sources


# Creating JSON data
json_result = {
    "input": result['input'],
    "sources": extract_list_sources(result),
    "answer": result['answer']
}

In [102]:
import json
from collections import OrderedDict

def extract_list_sources(result):
    sources = []
    for doc in result['context']:
        try:
            data = doc.metadata['metadata']
            data = json.loads(data)
            source = data.get('Source')
            if source:
                sources.append(source)
        except json.JSONDecodeError:
            # Handle the case where metadata is not a valid JSON
            continue
    sources = list(OrderedDict.fromkeys(sources))
    return sources

{'input': 'Quels sont les enseignements sur les investigations de la cellule 959B du HADE',
 'sources': ['C:\\Users\\gciprianherrera\\Desktop\\LLM\\MVP_Chatbot\\PDF\\HADE\\ELH-2021-008655.pdf',
  'C:\\Users\\gciprianherrera\\Desktop\\LLM\\MVP_Chatbot\\PDF\\HADE\\ELH-2019-036670.pdf',
  'C:\\Users\\gciprianherrera\\Desktop\\LLM\\MVP_Chatbot\\PDF\\HADE\\ELH-2019-073706.pdf',
  'C:\\Users\\gciprianherrera\\Desktop\\LLM\\MVP_Chatbot\\PDF\\STE2\\ELH-2023-026550.pdf',
  'C:\\Users\\gciprianherrera\\Desktop\\LLM\\MVP_Chatbot\\PDF\\HADE\\ELH-2019-029034.pdf'],
 'answer': "Le compte-rendu présente les résultats des investigations réalisées dans le pot florentin 239-74 et dans la cuve 239-60 de la cellule 959B du bâtiment HADE. Les objectifs de ces investigations étaient de prélever un échantillon de dépôt présent au fond du pot 239-74 et de définir un profil radiologique du pot, ainsi qu'un pigeage du liquide résiduel présent dans la cuve 239-60 et une vidéo à 360°. Les opérations d'investigati

In [1]:
import json
from collections import OrderedDict
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
import os
from langchain_community.retrievers import AzureAISearchRetriever


class Chatbot:
    def __init__(self):
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version="2023-05-15",
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
            model="gpt-35-turbo",
            temperature=0,
        )

        self.index_name = "complete_dataset"
        self.retriever = AzureAISearchRetriever(
            content_key="content", top_k=30, index_name=self.index_name
        )

        system_prompt = (
            "Use the given context to answer the question. "
            "If you don't know the answer, say you don't know. "
            "Use five sentences maximum and keep the answer concise. "
            "Context: {context}"
        )
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{input}"),
            ]
        )
        self.question_answer_chain = create_stuff_documents_chain(self.llm, prompt)
        self.chain = create_retrieval_chain(self.retriever, self.question_answer_chain)

    def ask(self, user_query):
        result = self.chain.invoke({"input": user_query})
        
        def extract_list_sources(result):
                sources = []
                for doc in result['context']:
                    try:
                        data = doc.metadata['metadata']
                        data = json.loads(data)
                        source = data.get('Source')
                        if source:
                            sources.append(source)
                    except json.JSONDecodeError:
                        # Handle the case where metadata is not a valid JSON
                        continue
                sources = list(OrderedDict.fromkeys(sources))  # Remove duplicates while preserving order
                return sources
        
        # Create the json_result
        json_result = {
            "input": result['input'],
            "sources": extract_list_sources(result),
            "answer": result['answer']
        }
    
        return json_result

In [2]:
from chat import Chatbot

chatbot = Chatbot()
# query = "Quel est l'objectif du compte-rendu de la cellule 951 DEG"  
# query = "Quel est l'objectif de l'investigation sur l’ensemble porte-lame cellule 904"  
# query = "Quels sont les équipements investigués sur le périmètre DEG"  
# query = "Quel est le débit de dose de la 951 du DEG " 
# query = "Quelles sont les informations de la cellule 909 du DEG " 
# query = "Quelle est la quantité de boue de la fosse 217-02 du DEG"  
# query = "DDD des PE de boues sur la fosse 217-02 du DEG " 
# query = "Quels sont les enseignements sur les investigations de la cellule 959B du HADE"
query = "Quelles sont les informations sur l'état de la cellule 929A du périmètre HADE"
response = chatbot.ask(query)
print(response['input'])
print(response['answer'])
response['sources']

Quelles sont les informations sur l'état de la cellule 929A du périmètre HADE
La cellule 929A a été investiguée en avril et mai 2021 dans le cadre d'une opération de détalutage et de prélèvements de dépôts. Des vidéos de la cellule ont été réalisées et sont disponibles. La cellule mesure 3,15m de largeur, 3,62m de longueur et 2,80m de hauteur. Des mesures de dépression et de débit de dose ont été réalisées, ainsi que des prélèvements de dépôts avec un outil GODET. Les prélèvements ont un aspect sableux. Des mesures de débits de dose ont également été réalisées dans la cellule.


'C:\\Users\\gciprianherrera\\Desktop\\LLM\\MVP_Chatbot\\PDF\\HADE\\ELH-2021-028061.pdf'